In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black
%load_ext Cython

In [ ]:
import gcsfs
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import nzthermo as nzt


plt.rcParams["figure.figsize"] = (12, 8)

plt.rcParams["xtick.bottom"] = False
plt.rcParams["ytick.left"] = False
plt.rcParams["xtick.labelbottom"] = False
plt.rcParams["ytick.labelleft"] = False

In [ ]:
fs = gcsfs.GCSFileSystem(token="anon")
mapper = fs.get_mapper("gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr")
ds = xr.open_zarr(mapper)
ds

In [ ]:
pressure = ds.coords["level"].to_numpy().astype(np.float32) * 100.0  # (Pa) (13,)
temperature = ds["temperature"].isel(time=slice(0, 30)).to_numpy().astype(np.float32)  # (K) (30, 13, 721, 1440)
specific_humidity = (
    ds["specific_humidity"].isel(time=slice(0, 30)).to_numpy().astype(np.float32)
)  # (K) (30, 13, 721, 1440)

In [ ]:
temperature[0, 0, ::12, 0]

In [ ]:
# - weatherbench's levels are in reverse order
# - non vertical dimensions are flattened like (T, Z, Y, X) -> (T*Y*X, Z) || (N, Z)
P = pressure[::-1]
Z = len(P)
T = np.moveaxis(temperature[:, ::-1, :, :], 1, -1).reshape(-1, Z)  # (N, Z)
print(f"{temperature.shape} -> {T.shape} || (T, Z, Y, X) -> (N, Z)")
Td = nzt.dewpoint_from_specific_humidity(
    P[np.newaxis, :],
    np.moveaxis(specific_humidity[:, ::-1, :, :], 1, -1).reshape(-1, Z),
)  # (N, Z)

In [ ]:
T0 = T[:, 0]
Td0 = Td[:, 0]

lr = nzt.moist_lapse(P, T0, Td0).reshape((temperature.shape[0],) + temperature.shape[2:] + (Z,))  # (T, Y, X, Z)
plt.imshow(lr[0, ..., 0])

In [ ]:
dcape = nzt.downdraft_cape(P, T, Td)  # (T*Y*X,) || (N,)
dcape = dcape.reshape((temperature.shape[0],) + temperature.shape[2:])  # (T, Y, X)
plt.imshow(dcape[0, ...])

In [ ]:
fig, axes = plt.subplots(dcape.shape[0] // 3, 3, figsize=(10, 20))
axes = axes.flatten()
for i, ax in enumerate(axes):
    ax.imshow(dcape[i], cmap="viridis")

In [ ]:
P0 = P[:1].repeat(T.shape[0])
T0 = T[:, 0]  # (T*Y*X)
Td0 = Td[:, 0]  # (T*Y*X)


wb = nzt.wet_bulb_temperature(P0, T0, Td0).reshape((temperature.shape[0],) + temperature.shape[2:])


plt.imshow(wb[0, ...])

In [ ]:
import nzthermo._c as c
import numpy as np
import metpy.calc as mpcalc
from metpy.units import units
import nzthermo as nzt

P = np.random.uniform(101325, 10000, 1000)
T = np.random.uniform(300, 200, 1000)
Td = T - np.random.uniform(0, 10, 1000)

lcl_p, lcl_t = nzt.lcl(P, T, Td)
lcl_p_, lcl_t_ = (x.m for x in mpcalc.lcl(P * units.Pa, T * units.kelvin, Td * units.kelvin))  # type: ignore
np.testing.assert_allclose(lcl_p, lcl_p_, rtol=1e-3)
np.testing.assert_allclose(lcl_t, lcl_t_, rtol=1e-3)


%timeit nzt.lcl(P, T, Td)
%timeit mpcalc.lcl(P * units.Pa, T * units.kelvin, Td * units.kelvin)

In [ ]:
P = np.random.uniform(101325, 10000, 1000).astype(np.float32)
T = np.random.uniform(300, 200, 1000).astype(np.float32)
Td = T - np.random.uniform(0, 10, 1000).astype(np.float32)

np.testing.assert_allclose(
    nzt.wet_bulb_temperature(P, T, Td),
    mpcalc.wet_bulb_temperature(P * units.Pa, T * units.kelvin, Td * units.kelvin).m,
    rtol=1e-3,
)


%timeit nzt.wet_bulb_temperature(P, T, Td)
%timeit mpcalc.wet_bulb_temperature(P * units.Pa, T * units.kelvin, Td * units.kelvin)

In [4]:
import nzthermo._c as c
import numpy as np
import nzthermo as nzt

pressure = np.linspace(100000, 31000, 20).astype(np.float32)#.reshape(1, -1)
temperature = np.random.uniform(300, 220, 20).astype(np.float32)
refrence_pressures = np.random.uniform(1001325, 100001, 20).astype(np.float32)
N = temperature.shape[0]
a = nzt.moist_lapse(pressure, temperature, refrence_pressures)
b = nzt.moist_lapse(pressure[np.newaxis,:], temperature, refrence_pressures)
print(
    f"""
>>> nzt.moist_lapse(pressure, temperature, refrence_pressures)

"""
)



[194.0076   122.54542  159.37715  178.9348   158.93712  172.7603
 201.70447  122.40376  130.10341  124.225586 143.72778  198.83835
 130.11867  158.52783  141.45505  119.412865 143.37851  124.279175
  90.33691  121.02936 ]
